In [312]:
import pandas as pd
import numpy as np
import random

#normalizing each attribute so no one attribute dominates the classification
def normalize(data):
    normdata=pd.DataFrame(index=data.index.tolist(),columns = data.columns.tolist())
    for (columname, columnData) in data.iteritems():
        if columname!='clss':
            normdata[columname] = [(data.loc[index,columname]-data[columname].min())/
                                   (data[columname].max()-data[columname].min()) for index,row in data.iterrows()]
        else:
            normdata[columname]=data[columname]
    
    return normdata

#euclidean distance between each example in the training set and the new example we want to classify
def distance(tset,v):
    tset['dist']=0
    for index,row in tset.iterrows():
        tset.loc[index,'dist']=np.linalg.norm(tset.iloc[index,:-2].to_numpy()-v.iloc[0].to_numpy())
    return tset

#puts the k nearest neigbors by choosing the class that comes up the most times in the k smallest distances  
def classify(tset,k,v):
    normtset=normalize(tset)
    normtset = distance(normtset,v)
    nn=normtset.nsmallest(k,'dist')
    clss = pd.DataFrame(nn['clss'].value_counts())
    
    return "most probable class is "+ str(clss.index[0])
        

In [319]:
#example training set from chapter 1 (or 2 i forgot)
z=pd.read_csv("example1.csv",header=0)
z=z.rename(columns ={'class':'clss'})
v = pd.DataFrame({'at1': [3.1], 'at2': [2], 'at3': [1.99]})

classify(z,3,v)

'most probable class is pos'

In [320]:
#training set from uci repository
d = pd.read_csv('wines.csv')
d = d.rename(columns ={'class':'clss'})
#next two lines are me moving the class of each example to the end of the df bc its easier
#theres obviously a way to classify/get distance without changing the df but idgaf i already did it this way

t1=d.pop('clss')
d['clss']=t1
n=normalize(d)

#creating an example we want to classify
test=pd.DataFrame(index=[0],columns=n.columns.tolist())
test=test.fillna(0)

for (columname,columnData) in n.iteritems():
    test[columname]=round(random.uniform(n[columname].min(),n[columname].max()),5)

test=test.drop(columns='clss')
classify(d,5,test)


'most probable class is 2'